In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

teams = pd.read_csv("../data/march_madness_2024.csv")

In [9]:
from march_madness.tournament import Tournament

tournament = Tournament(teams=teams)

unplayed_games = tournament.get_unplayed_games()
print(f"Round 1: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)

unplayed_games = tournament.get_unplayed_games()
print(f"Round 2: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)
    
unplayed_games = tournament.get_unplayed_games()
print(f"Round 3: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)
    
unplayed_games = tournament.get_unplayed_games()
print(f"Round 4: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)

unplayed_games = tournament.get_unplayed_games()
print(f"Round 5: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1)
    
unplayed_games = tournament.get_unplayed_games()
print(f"Round 6: {len(unplayed_games)} games")
for game in unplayed_games:
    tournament.update_game_result(game, game.team1, verbose=True)
    

Round 1: 32 games
Round 2: 16 games
Round 3: 8 games
Round 4: 4 games
Round 5: 2 games
Round 6: 1 games


In [7]:
from march_madness.prediction import random_prediction, team1_always_wins
from march_madness.tournament import Tournament, TournamentSimulator

simulator = TournamentSimulator(num_trials=1000,
                                tournament_class=Tournament,
                                tournament_params={"teams": teams},
                                prediction_strategy=random_prediction,
                                prediction_details=True,
                                seed=42)

simulator.run(verbose=True)
df = simulator.to_dataframe()
df.head()

  0%|          | 0/1000 [00:00<?, ?it/s]

,trial,round,team1,team2,winner,game_id,tournament_winner,prediction_details,team1_details,team2_details,winner_details,tournament_state
0,1,1,Houston,Longwood,Longwood,"{'round': 1, 'region': 'South', 'game': 1}",None,{'confidence': 0.5},"{'team': 'Houston', 'seed': 1, 'region': 'South'}","{'team': 'Longwood', 'seed': 16, 'region': 'So...","{'team': 'Longwood', 'seed': 16, 'region': 'So...","[(1, Houston, Longwood, Longwood)]"
1,1,1,Nebraska,Texas A&M,Nebraska,"{'round': 1, 'region': 'South', 'game': 2}",None,{'confidence': 0.5},"{'team': 'Nebraska', 'seed': 8, 'region': 'Sou...","{'team': 'Texas A&M', 'seed': 9, 'region': 'So...","{'team': 'Nebraska', 'seed': 8, 'region': 'Sou...","[(1, Houston, Longwood, Longwood), (1, Nebrask..."
2,1,1,Wisconsin,James Madison,James Madison,"{'round': 1, 'region': 'South', 'game': 3}",None,{'confidence': 0.5},"{'team': 'Wisconsin', 'seed': 5, 'region': 'So...","{'team': 'James Madison', 'seed': 12, 'region'...","{'team': 'James Madison', 'seed': 12, 'region'...","[(1, Houston, Longwood, Longwood), (1, Nebrask..."
3,1,1,Duke,Vermont,Vermont,"{'round': 1, 'region': 'South', 'game': 4}",None,{'confidence': 0.5},"{'team': 'Duke', 'seed': 4, 'region': 'South'}","{'team': 'Vermont', 'seed': 13, 'region': 'Sou...","{'team': 'Vermont', 'seed': 13, 'region': 'Sou...","[(1, Houston, Longwood, Longwood), (1, Nebrask..."
4,1,1,Texas Tech,NC State,Texas Tech,"{'round': 1, 'region': 'South', 'game': 5}",None,{'confidence': 0.5},"{'team': 'Texas Tech', 'seed': 6, 'region': 'S...","{'team': 'NC State', 'seed': 11, 'region': 'So...","{'team': 'Texas Tech', 'seed': 6, 'region': 'S...","[(1, Houston, Longwood, Longwood), (1, Nebrask..."


In [8]:
df["game_id_str"] = df["game_id"].astype(str)

trial_brackets = df.groupby("trial")[["game_id_str", "winner"]].apply(lambda x: tuple(zip(x["game_id_str"], x["winner"])))

bracket_counts = trial_brackets.value_counts().reset_index()
bracket_counts.columns = ["bracket", "count"]

total_trials = df["trial"].nunique()
bracket_counts["percentage"] = (bracket_counts["count"] / total_trials) * 100

# most common full bracket
best_bracket_tuple = bracket_counts.iloc[0]["bracket"]  # Most frequent bracket
best_bracket_df = pd.DataFrame(best_bracket_tuple, columns=["game_id_str", "winner"])

# Step 7: Add frequency and percentage for each game
game_frequencies = df.groupby(["game_id_str", "winner"]).size().reset_index(name="game_count")
game_frequencies["game_percentage"] = (game_frequencies["game_count"] / total_trials) * 100

best_bracket_df = best_bracket_df.merge(game_frequencies, on=["game_id_str", "winner"], how="left")

best_bracket_df = best_bracket_df.drop(columns=["game_id_str"])

best_bracket_df

# weight later rounds heavier?

,winner,game_count,game_percentage
0,Longwood,485,48.5
1,Nebraska,492,49.2
2,James Madison,517,51.7
3,Vermont,547,54.7
4,Texas Tech,469,46.9
...,...,...,...
58,Northwestern,66,6.6
59,McNeese,62,6.2
60,Alabama,27,2.7
61,McNeese,33,3.3
